<br>
<a href="https://www.nvidia.cn/training/online/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用 NVIDIA Earth-2 部署 AI 天气模型</h1>
<h2><b>练习 2：</b> 验证</h2>
<br>

课程的第二部分，我们将更仔细地查看 FourCastNet (SFNO) 生成的输出数据。验证气象模型输出时，不同的应用有不同的目标。例如，能源行业的应用可能对日内和前一天的风力发电预测感兴趣，而农业的应用可能更关注季节性的时间框架。我们将探索一组常见的评估指标，帮助理解预测如何增加价值。请记住，这里只处理一个小数据集作为演示。然而，严格的验证通常会覆盖多个起始时间和更多的集成成员。

In [ ]:
import torch
import xarray as xr

from earth2studio.data import NCAR_ERA5, WB2Climatology, fetch_data, prep_data_array
from earth2studio.statistics import acc, crps, lat_weight, rank_histogram, rmse, spread_skill_ratio
from plot import animate_global, animate_local, figure_global, plot_global_ensemble, plot_line_ensemble, plot_rank_examples, plot_rank_histograms, plot_score_line, plot_wind_power_output
from utils import get_lat_lon
from wind import calculate_power_output, get_wind_turbine_model

%matplotlib inline

## 探索

接下来将开始加载之前生成的预测。记得我们在上一部分保存了七个不同的变量。可以随意查看下面任何图表中的其它变量。

In [ ]:
ds = xr.open_dataset("/workspace/exercises/output/exercise_01.nc", engine="netcdf4")

start_time = ds.time.values.squeeze()
print("Available variables: ", list(ds))

先来创建全球 2 米温度的动画来获得一个概览。制作动画会花一些时间，但之后您可以观看小短片来查看天气预测。请注意，由于时间步骤之间是 6 小时的间隔，因此进度并不是特别平滑。Earth-2 还提供了一种 AI 插值模型，可以在模型输出之间提供额外的时间步骤，以实现更高的时间分辨率。

In [ ]:
animate_global(ds.isel(ensemble=0).t2m, start_time, max_frames=5)

地球上每时每刻都在发生很多事，所以我们来聚焦到一个更小的窗口。输出网格是 721x1440，这样就可以在一个四分之一度网格上选择纬度和经度（即，步长为 0.25）。出于本课程的目的，某些大城市的坐标被预先定义，并可通过 `get_lat_lon` 获取。先来看一下墨西哥湾周围的区域，该地区受到飓风贝瑞的严重影响。由于我们保存了 10 米风的 u 和 v 分量，因此可以通过取欧几里得长度（平方和的平方根）来计算每个点的风速。这样就能清楚地看到飓风朝着美国南部边界前进。

In [ ]:
location = "Houston"
loc_lat, loc_lon = get_lat_lon(location)

In [ ]:
animate_local((ds.isel(ensemble=0).u10m ** 2 + ds.isel(ensemble=0).v10m ** 2) ** 0.5, start_time, loc_lat, loc_lon, cb_label="Wind speed [m/s]")

上面的图只显示了第一个集成成员的输出。我们可以通过查看集成成员之间的差异更好地了解预测的不确定性。为此，我们将首先聚焦在一个网格单元上。下面图中的每条线都显示了一个集成成员的轨迹。与后期的集成扩展相比，初始扰动是很小的。这种对初始条件微小变化的敏感性就是我们说天气系统是 [chaotic](https://en.wikipedia.org/wiki/Chaos_theory) 的原因。

In [ ]:
plot_line_ensemble(ds.tcwv, start_time, loc_lat, loc_lon, "tcwv [kg/m²]")

下面的图展示了更大的背景。左侧的地球仪展示了在预测最后一步的三个集成成员的输出。右侧的地球仪显示了成员的标准差，可以识别出高变异性和低变异性的区域。

In [ ]:
plot_global_ensemble(ds.tcwv, start_time, loc_lat, loc_lon, cb_label="tcwv [kg/m²]", ilead_time=10)

我们希望利用已经做出的预测来判断一个风电场的电力输出。先从小处着手，计算一下风轮的预期电力输出。为此，使用 `windpowerlib`，这是一个专注于风力发电建模的 Python 库。可以从众多预定义参考配置中选一个，并将其传递给库，库会根据 10 米和 100 米风速处理计算。

In [ ]:
# Here, we use the IEA 15 MW offshore reference turbine
# Supported turbine types can be found at https://openenergyplatform.org/dataedit/view/supply/wind_turbine_library
wind_turbine_specification = {
    "turbine_type": "IEA-15-240-RWT",  # turbine type as in register
    "hub_height": 150,  # in m
}
wt = get_wind_turbine_model(wind_turbine_specification)

电力输出是基于使用估计粗糙度长度的对数剖面推断的风速来计算的。图中显示的是轮毂高度的风速、风轮的功率曲线，以及预测的电力输出随预测时间的变化。每个集成成员以虚线表示，而实线则表示集成均值。

In [ ]:
ds_power = calculate_power_output(ds, wt, loc_lat=65, loc_lon=23)
plot_wind_power_output(ds_power, wt, start_time=start_time)

## 评分

为了评估预测的准确性，我们将输出与官方的 ERA5 再分析结果进行评分。Earth2Studio 提供了优化的例程来计算评分指标，但首先需要获取参考数据进行比较。为此，可以使用 `fetch_data`，它在运行集成工作流时也被用到了。像工作流一样，它需要一个数据源（这里再次使用 `NCAR_ERA5`）、一个开始时间、请求的变量和预测时间。对于工作流，只需要一个时间切片作为初始条件，但对于评分，将需要使用所有 17 个时间切片来比较时间上的准确性。

In [ ]:
ncar_era5 = NCAR_ERA5(verbose=True)
scoring_variables = ["t2m", "u10m", "z500"]

x_true, coords_true = fetch_data(
    source=ncar_era5,
    time=[start_time],
    variable=scoring_variables,
    lead_time=ds.lead_time.values,
)

Earth2Studio 提供了 `prep_data_array` 便利函数，用于将 xarray 的 `DataArray` 转换为纯 PyTorch 张量及其相应的坐标。通过 PyTorch 张量，可以利用 GPU 来加速计算。而这些坐标则跟踪维度的信息。

In [ ]:
x_pred, coords_pred = prep_data_array(ds[scoring_variables].to_dataarray().transpose("ensemble", "time", "lead_time", ...))

因为我们正在处理一个常规的 721x1440 网格，所以在计算全球验证分数时，应降低极地点的权重。这是因为在更高纬度区域，个别点之间的距离小于赤道附近的距离。如果不加权重，就会加大极地附近地区的重要性。可以使用 Earth2Studio 的 `lat_weight` 为极地地区分配逐渐减小的权重。

In [ ]:
weights = torch.ones(len(ds.lon)) * lat_weight(torch.from_numpy(ds.lat.values)).unsqueeze(1)

figure_global(xr.DataArray(data=weights, coords={"lat": ds.lat.values, "lon": ds.lon.values}), *get_lat_lon("San Jose"), colorbar=True)

在给概率集成评分时，我们使用的指标与给确定性预测评分时不同。然而，通常的做法是对集成均值进行确定性验证，因此首先会这么做。一个简单且常用的确定性评分指标是均方根误差（RMSE），它计算预测值与参考数据之间的平方差，将所有数据点的差值相加，然后开方。要计算 RMSE，只需要将输入张量和权重传递给 Earth2Studio 提供的 `rmse` 评分例程即可。我们指定 `lat` 和 `lon` 为归约维度，这样就能每个变量和每个时间步返回一个全局值。因此，提前提供纬度权重是很重要的。评分函数还会返回一个 PyTorch 张量及其坐标，解包返回值并让 xarray 根据结果构建数据集，这样就更方便绘图了。

In [ ]:
RMSE = rmse(reduction_dimensions=["lat", "lon"], weights=weights)
rmse_score = xr.DataArray(*RMSE(x_pred.mean(axis=0), {k: v for k, v in coords_pred.items() if k != "ensemble"}, x_true, coords_true)).to_dataset("variable")

在下面的图中，我们可以看到 RMSE 在短期预测时较小，随后随着预测时间的延长而逐渐增加。这是意料之中的，因为模型过程中的不确定性会随着时间的推移而叠加。预测时间越长，预测的准确性通常越低。

In [ ]:
plot_score_line(rmse_score, start_time, ["t2m", "u10m", "z500"], ["RMSE [K]", "RMSE [m/s]", "RMSE [gpm]"])

集成均值通常比单一的确定性预测更能有效估计未来情况。然而，只有当集成成员之间的偏差覆盖所有可能的未来天气场景时，集成才有意义。如果成员之间过于相似，我们就会低估天气极端事件的发生。此时，集成被称为“欠分散的”。如果差异过大，则集成被称为“过分散的”。那么，集成成员之间的分散程度应该有多大呢？一个设计用来回答这个问题的指标是``Spread-Skill Ratio (SSR)``。它将分散度（集成成员的标准差）与技能（集成的 RMSE）进行比较。理想情况下，两者的比率应为 1。可以像计算 RMSE 一样使用 ``spread_skill_ratio`` 来计算这个比率，唯一的区别是这次还需要指定集成维度的名称，这样过程才能知道在哪个维度上计算均值和标准差。

In [ ]:
SSR = spread_skill_ratio(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], reduction_weights=weights)
ssr = xr.DataArray(*SSR(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

下面的折线图显示了三个变量的 SSR。大于 1 的值意味着集成是过分散的，而小于 1 的值则表示它是欠分散的。SRR 在短期预报中似乎表现不佳，这表明应该花点时间来调整我们的扰动方法。之前提到的 LBNL 和 NVIDIA 研究人员的两篇论文介绍了一种基于繁殖向量的扰动方法，能有效影响 AI 气象预报（[第 1 部分](https://arxiv.org/abs/2408.03100)和[第 2 部分](https://arxiv.org/abs/2408.01581v1)）。

In [ ]:
plot_score_line(ssr, start_time, ["t2m", "u10m", "z500"], ["SSR", "SSR", "SSR"])

第二个重要的概率预报验证策略是使用排名直方图。对于这些，我们不计算单一的分数，而是查看验证数据与集成成员相比的排名。排名是验证值在所有集成值与验证值一起排序时所处的位置。例如，如果有 13 个集成成员预测的值小于验证值，那么验证排名将是 14。对每一个预测都这样做，然后查看这些排名的直方图。

理想情况下，这些排名应该均匀分布，这说明验证结果与集成预测无法区分。对于欠分散的集成，验证排名通常非常高或低，形成 U 型直方图。在过分散的情况下，排名会集中在中间，因为集成预测比验证更极端（更高或更低）。还可以识别偏差。负偏差表现为向上倾斜，因为验证通常会获得高排名。正偏差则表现为向下倾斜。

In [ ]:
plot_rank_examples()

计算排名直方图的方式和其它指标差不多。不同的是，结果数据包含了关于直方图桶中心和计数的信息，我们可以进行绘图。

In [ ]:
RH = rank_histogram(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], number_of_bins=5)
rh = xr.DataArray(*RH(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

将结果直方图与上面的例子进行比较。不同的变量在不同的前置时间下表现如何？

In [ ]:
plot_rank_histograms(rh, start_time, ["t2m", "u10m", "z500"], appr_n=4)

再次提醒，我们仅针对单一的起始时间和一个小的集成计算了统计数据。在实际操作中，这一评估会针对多个起始时间和/或更长的预报时间范围，并且使用更大的集成。此外，使用不同类型的验证数据也可能有意义。例如，我们使用了再分析数据。其他选项包括实际观测值、其他预报数据或气候资料。

完成此 ``Notebook`` 后，请关闭内核（在顶部的上下文菜单中选择“Kernel”，然后选择“Shut Down Kernel”）。接着前往[练习 3 的 Notebook](./exercise_03_downscaling.ipynb) 学习降尺度。如果您还有空余时间，可以下面了解再多两个常见的评估指标。

## 额外内容

另一个常用的确定性预测评估指标是[异常相关系数](https://confluence.ecmwf.int/display/FUG/Section+6_2_2+Anomaly+Correlation+Coefficient)（ACC）。它是预报与参考之间的异常相关性，都是相对气候而言。其值介于 -1 和 1 之间，1 表示完美相关。Earth2Studio 提供了来自 [WeatherBench 2](https://weatherbench2.readthedocs.io/en/latest/data-guide.html) 的气候数据的便捷访问。我们只需将 ``WB2Climatology`` 数据接口传递给评分例程，它会自动下载并缓存所需的数据。除了指定气候外，计算 RMSE 的代码是一样的。

In [ ]:
ACC = acc(reduction_dimensions=["lat", "lon"], climatology=WB2Climatology(), weights=weights)
acc_score = xr.DataArray(*ACC(x_pred.mean(axis=0), {k: v for k, v in coords_pred.items() if k != "ensemble"}, x_true, coords_true)).to_dataset("variable")

在预报的早期，预测 ACC 值接近 1，表明表现非常好。随着预报的进行，ACC 逐渐下降。我们认为，ACC 值在 0.55-0.6 以上的预报是有价值的（例如，参见[德国气象局](https://www.dwd.de/EN/ourservices/seasonals_forecasts/anomaly_correlation.html)和 [ECWMF](https://confluence.ecmwf.int/display/FUG/Section+6_2_2+Anomaly+Correlation+Coefficient)）。这意味着在这次评估的 96 小时内，预报是有价值的。

In [ ]:
plot_score_line(acc_score, start_time, ["t2m", "u10m", "z500"], ["ACC", "ACC", "ACC"])

另一个重要的评估指标是连续排名概率分数（CRPS）。它用于衡量预报概率分布与真实结果之间的差异。CRPS 分数是通过计算预报的累积分布函数（CDF）与真实 CDF 之间区域的积分来得到的。分数越低，预报越好。对于确定性预报，它默认为平均绝对误差（MAE）。利用 Earth2Studio，我们可以像计算其它指标一样计算 CRPS。

In [ ]:
CRPS = crps(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], weights=weights)
crps_score = xr.DataArray(*CRPS(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

正如预期的那样，CRPS 一开始较低，然后在较长的前置时间内逐渐增加。

In [ ]:
plot_score_line(crps_score, start_time, ["t2m", "u10m", "z500"], ["CRPS", "CRPS", "CRPS"])